In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os

# Solicitar el archivo txt al usuario
from google.colab import files

print("Por favor, sube el archivo .txt con las intervenciones de Pedro Sanchez.")
uploaded = files.upload()

# Leer el archivo .txt
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = text.replace('\n', ' ')  # Reemplazar saltos de línea por espacios
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    return text

text = preprocess_text(text)
words = text.split()  # Dividir el texto en palabras únicas
vocab = sorted(set(words))  # Crear vocabulario único

# Crear diccionarios de mapeo
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convertir el texto a índices
encoded_text = [word_to_idx[word] for word in words]

# Preparar los datos para el modelo
sequence_length = 10  # Longitud de las secuencias
X = []
y = []

for i in range(len(encoded_text) - sequence_length):
    X.append(encoded_text[i:i+sequence_length])
    y.append(encoded_text[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Construir el modelo LSTM con 3 capas
vocab_size = len(vocab)
embedding_dim = 128  # Dimensión del embedding
lstm_units = 512  # Número de unidades en LSTM

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    LSTM(lstm_units, return_sequences=True),
    Dropout(0.3),
    LSTM(lstm_units, return_sequences=True),
    Dropout(0.3),
    LSTM(lstm_units),
    Dropout(0.3),
    Dense(vocab_size, activation='softmax')
])

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

# Guardar el mejor modelo durante el entrenamiento
checkpoint = ModelCheckpoint('modelo_LSTM_3capas_Sanchez.keras', monitor='loss', save_best_only=True)

# Entrenar el modelo con un número moderado de épocas
print("Entrenando el modelo...")
model.fit(X, y, epochs=100, batch_size=128, verbose=1, callbacks=[checkpoint])

# Función para generar texto con control de temperatura
def generate_text(model, start_sequence, max_words=150, temperature=1.0):
    input_sequence = [word_to_idx[word] for word in start_sequence.split() if word in word_to_idx]
    generated = start_sequence

    for _ in range(max_words):
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], maxlen=sequence_length)
        predictions = model.predict(input_padded, verbose=0).flatten()
        predictions = np.log(predictions + 1e-9) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        predicted_idx = np.random.choice(len(probabilities), p=probabilities)
        predicted_word = idx_to_word[predicted_idx]

        generated += ' ' + predicted_word
        input_sequence.append(predicted_idx)
        input_sequence = input_sequence[1:]

    return generated

# Interactuar con el modelo generando texto
print("¡Modelo entrenado! Puedes interactuar con él para generar texto.")
while True:
    prompt = input("Introduce el inicio del texto (o escribe 'salir' para terminar): ")
    if prompt.lower() == 'salir':
        print("¡Hasta luego!")
        break
    print("Generando texto...")
    print(generate_text(model, prompt, max_words=150, temperature=0.8))


Por favor, sube el archivo .txt con las intervenciones de Pedro Sanchez.


Saving intervencionesSanchez.txt to intervencionesSanchez.txt


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Entrenando el modelo...
Epoch 1/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.0521 - loss: 7.2264
Epoch 2/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0577 - loss: 6.2020
Epoch 3/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0564 - loss: 6.1627
Epoch 4/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0570 - loss: 6.1839
Epoch 5/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0586 - loss: 6.1668
Epoch 6/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0619 - loss: 6.1470
Epoch 7/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0597 - loss: 6.1575
Epoch 8/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0610 - loss: 6.1433
Epoch 9/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.0631 - loss: 6.1221
Epoch 10/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0639 - loss: 6.1272
Epoch 11/100
308/308 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.0666 - loss: 6.0786
Epoch